In [1]:
import pandas as pd

In [2]:
# Load the CSV file
FA = '../FAresults.xlsx'
CV = '../CVresults.xlsx'
static2SSP = '../static2SSPresults.xlsx'
rolling2SSP = '../rolling2SSPresults.xlsx'
WS = '../WSresults.xlsx'

df_FA = pd.read_excel(FA)
df_CV = pd.read_excel(CV)
df_s2SSP = pd.read_excel(static2SSP)
df_r2SSP = pd.read_excel(rolling2SSP)
df_WS = pd.read_excel(WS)


In [3]:
if df_CV.shape[0] != df_FA.shape[0] or df_CV.shape[0] != df_s2SSP.shape[0] or df_CV.shape[0] != df_r2SSP.shape[0] or df_CV.shape[0] != df_WS.shape[0]:
    print("ERROR! # of rows in the data frames are inconsistent!")
else:
    print("df_CV.shape = ", df_CV.shape)

df_CV.shape =  (108, 12)


In [4]:
# Do separate tables for different instance_option (-1: deterministic, 0: random) and cost_structure (0: old,1: safetime)

In [5]:
def createTable(i_option, c_structure, outputfile):
    df_CV_0 = df_CV[(df_CV['instance_option'] == i_option) & (df_CV['cost_structure'] == c_structure)]
    df_FA_0 = df_FA[(df_FA['instance_option'] == i_option) & (df_FA['cost_structure'] == c_structure)]
    df_s2SSP_0 = df_s2SSP[(df_s2SSP['instance_option'] == i_option) & (df_s2SSP['cost_structure'] == c_structure)]
    df_r2SSP_0 = df_r2SSP[(df_r2SSP['instance_option'] == i_option) & (df_r2SSP['cost_structure'] == c_structure)]
    df_WS_0 = df_WS[(df_WS['instance_option'] == i_option) & (df_WS['cost_structure'] == c_structure)]
    df_CV_0.reset_index(drop=True, inplace=True)
    df_FA_0.reset_index(drop=True, inplace=True)
    df_s2SSP_0.reset_index(drop=True, inplace=True)
    df_r2SSP_0.reset_index(drop=True, inplace=True)
    df_WS_0.reset_index(drop=True, inplace=True)
    if c_structure == 0:
        # table macros for this case
        tauList = [0, 0.5, 5]
        NiList = [3,6,9]
        NjList = [10,20,30]
    elif c_structure == 1:
        tauList = [2, 5, 10]
        NiList = [3,6,9]
        NjList = [10,20,30]
    
    # Create a LaTeX table string with the specified columns and multi-row header
    latex_table_custom = '';
    
    idx = 0;
    for i1 in range(len(tauList)):
        for i2 in range(len(NiList)):
            for i3 in range(len(NjList)):
                CV_val = df_CV_0["obj"][idx] 
                CV_CI = df_CV_0["CI"][idx] 
                FA_time = df_FA_0["train_time"][idx]
                FA_gap = int((df_FA_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                WS_time = df_WS_0["train_time"][idx]
                WS_gap = int((df_WS_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                r2SSP_gap = int((df_r2SSP_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)
                s2SSP_time = df_s2SSP_0["train_time"][idx]
                s2SSP_gap = int((df_s2SSP_0["obj"][idx]-df_CV_0["obj"][idx])/df_CV_0["obj"][idx]*100)


                if i2 == 0:
                    if i3 == 0:
                        latex_table_custom += f"{tauList[i1]} & {NiList[i2]} $\quad$ {NjList[i3]} && {round(CV_val,1)} & {round(CV_CI,1)} && {round(FA_time,1)} & {FA_gap}\\% && {round(WS_time,1)} & {WS_gap}\\% && {r2SSP_gap}\\% && {round(s2SSP_time,1)} & {s2SSP_gap}\\% \\\\ \n"
                    else:
                        latex_table_custom += f" &  $\quad\quad$ {NjList[i3]} && {round(CV_val,1)} & {round(CV_CI,1)} && {round(FA_time,1)} & {FA_gap}\\% && {round(WS_time,1)} & {WS_gap}\\% && {r2SSP_gap}\\% && {round(s2SSP_time,1)} & {s2SSP_gap}\\%  \\\\ \n"
                else:
                    if i3 == 0:
                        latex_table_custom += f" & {NiList[i2]}  $\quad\quad$ {NjList[i3]} && {round(CV_val,1)} & {round(CV_CI,1)} && {round(FA_time,1)} & {FA_gap}\\% && {round(WS_time,1)} & {WS_gap}\\% && {r2SSP_gap}\\% && {round(s2SSP_time,1)} & {s2SSP_gap}\\%  \\\\ \n"
                    else:
                        latex_table_custom += f" &  $\quad\quad$ {NjList[i3]} && {round(CV_val,1)} & {round(CV_CI,1)} && {round(FA_time,1)} & {FA_gap}\\% && {round(WS_time,1)} & {WS_gap}\\% && {r2SSP_gap}\\% && {round(s2SSP_time,1)} & {s2SSP_gap}\\%  \\\\ \n"
                idx += 1;
        if i1 != len(tauList)-1:
            latex_table_custom += "\\hline\n"

    print(latex_table_custom)        
    # Save to a new .tex file
    with open(outputfile, 'w') as f:
        f.write(latex_table_custom)

In [6]:
createTable(-1,0,'Table-synthetic-gap-summary1.tex')
createTable(0,0,'Table-synthetic-gap-summary2.tex')
createTable(-1,1,'Table-synthetic-gap-summary3.tex')
createTable(-1,1,'Table-synthetic-gap-summary4.tex')

0 & 3 $\quad$ 10 && 710.8 & 93.0 && 28.0 & 131\% && 2.8 & 131\% && 131\% && 0.0 & 753\% \\ 
 &  $\quad\quad$ 20 && 1038.2 & 139.3 && 41.8 & 271\% && 3.3 & 343\% && 464\% && 0.0 & 597\%  \\ 
 &  $\quad\quad$ 30 && 2874.1 & 420.6 && 37.5 & 200\% && 4.5 & 208\% && 207\% && 0.0 & 208\%  \\ 
 & 6  $\quad\quad$ 10 && 1421.6 & 179.0 && 28.5 & 115\% && 4.6 & 115\% && 115\% && 0.0 & 696\%  \\ 
 &  $\quad\quad$ 20 && 2065.0 & 262.6 && 42.8 & 251\% && 5.5 & 324\% && 488\% && 0.0 & 573\%  \\ 
 &  $\quad\quad$ 30 && 5616.1 & 745.6 && 38.6 & 196\% && 7.7 & 207\% && 204\% && 0.1 & 207\%  \\ 
 & 9  $\quad\quad$ 10 && 2123.2 & 267.0 && 26.5 & 123\% && 6.2 & 121\% && 121\% && 0.1 & 707\%  \\ 
 &  $\quad\quad$ 20 && 3031.4 & 382.3 && 50.7 & 262\% && 7.0 & 333\% && 506\% && 0.1 & 582\%  \\ 
 &  $\quad\quad$ 30 && 7829.0 & 1008.8 && 30.4 & 210\% && 11.1 & 223\% && 219\% && 0.1 & 223\%  \\ 
\hline
0.5 & 3 $\quad$ 10 && 1093.4 & 286.9 && 22.1 & 79\% && 4.2 & 79\% && 79\% && 0.0 & 462\% \\ 
 &  $\quad\quad$ 2